## SCD Implementation


In [0]:
CREATE DATABASE sales_scd;

In [0]:
CREATE TABLE sales_scd.orders (
    OrderID INT PRIMARY KEY,
    OrderDate DATE,
    CustomerID INT,
    CustomerName VARCHAR(100),
    CustomerEmail VARCHAR(150),
    ProductID INT,
    ProductName VARCHAR(100),
    ProductCategory VARCHAR(50),
    RegionID INT,
    RegionName VARCHAR(50),
    Country VARCHAR(50),
    Quantity INT,
    UnitPrice DECIMAL(10, 2),
    TotalAmount DECIMAL(12, 2)
);

In [0]:
INSERT INTO sales_scd.Orders (
    OrderID,
    OrderDate,
    CustomerID,
    CustomerName,
    CustomerEmail,
    ProductID,
    ProductName,
    ProductCategory,
    RegionID,
    RegionName,
    Country,
    Quantity,
    UnitPrice,
    TotalAmount
)
VALUES
(1, '2024-02-01', 101, 'Alice Johnson', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'USA', 2, 800, 1600),
(2, '2024-02-02', 102, 'Bob Smith', 'bob@example.com', 202, 'Smartphone', 'Electronics', 302, 'Europe', 'Germany', 1, 500, 500),
(3, '2024-02-03', 103, 'Charlie Brown', 'charlie@example.com', 203, 'Tablet', 'Electronics', 303, 'Asia', 'India', 3, 300, 900),
(4, '2024-02-04', 101, 'Alice Johnson', 'alice@example.com', 204, 'Headphones', 'Accessories', 301, 'North America', 'USA', 1, 150, 150),
(5, '2024-02-05', 104, 'David Lee', 'david@example.com', 205, 'Gaming Console', 'Electronics', 302, 'Europe', 'France', 1, 400, 400),
(6, '2024-02-06', 102, 'Bob Smith', 'bob@example.com', 206, 'Smartwatch', 'Electronics', 303, 'Asia', 'China', 2, 200, 400),
(7, '2024-02-07', 105, 'Eve Adams', 'eve@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'Canada', 1, 800, 800),
(8, '2024-02-08', 106, 'Frank Miller', 'frank@example.com', 207, 'Monitor', 'Accessories', 302, 'Europe', 'Italy', 2, 250, 500),
(9, '2024-02-09', 107, 'Grace White', 'grace@example.com', 208, 'Keyboard', 'Accessories', 303, 'Asia', 'Japan', 3, 100, 300),
(10, '2024-02-10', 104, 'David Lee', 'david@example.com', 209, 'Mouse', 'Accessories', 301, 'North America', 'USA', 1, 50, 50);

num_affected_rows,num_inserted_rows
10,10


## SCD TYPE - 1 

In [0]:
SELECT * FROM sales_scd.Orders;

OrderID,OrderDate,CustomerID,CustomerName,CustomerEmail,ProductID,ProductName,ProductCategory,RegionID,RegionName,Country,Quantity,UnitPrice,TotalAmount
1,2024-02-01,101,Alice Johnson,alice@example.com,201,Laptop,Electronics,301,North America,USA,2,800.00,1600.00
2,2024-02-02,102,Bob Smith,bob@example.com,202,Smartphone,Electronics,302,Europe,Germany,1,500.00,500.00
3,2024-02-03,103,Charlie Brown,charlie@example.com,203,Tablet,Electronics,303,Asia,India,3,300.00,900.00
4,2024-02-04,101,Alice Johnson,alice@example.com,204,Headphones,Accessories,301,North America,USA,1,150.00,150.00
5,2024-02-05,104,David Lee,david@example.com,205,Gaming Console,Electronics,302,Europe,France,1,400.00,400.00
6,2024-02-06,102,Bob Smith,bob@example.com,206,Smartwatch,Electronics,303,Asia,China,2,200.00,400.00
7,2024-02-07,105,Eve Adams,eve@example.com,201,Laptop,Electronics,301,North America,Canada,1,800.00,800.00
8,2024-02-08,106,Frank Miller,frank@example.com,207,Monitor,Accessories,302,Europe,Italy,2,250.00,500.00
9,2024-02-09,107,Grace White,grace@example.com,208,Keyboard,Accessories,303,Asia,Japan,3,100.00,300.00
10,2024-02-10,104,David Lee,david@example.com,209,Mouse,Accessories,301,North America,USA,1,50.00,50.00


## Steps for SCD
- In order to implement SCD, we first have to create view.

In [0]:
CREATE VIEW sales_scd.view_DimProducts 
AS
SELECT DISTINCT(ProductID) AS ProductID, ProductName, ProductCategory FROM sales_scd.orders;

In [0]:
CREATE OR REPLACE TABLE sales_scd.DimProducts
(
  ProductID INT,
  ProductName STRING,
  ProductCategory STRING
);

In [0]:
INSERT INTO sales_scd.DimProducts 
SELECT ProductID, ProductName, ProductCategory FROM sales_scd.view_DimProducts;

num_affected_rows,num_inserted_rows
9,9


In [0]:
SELECT * FROM sales_scd.DimProducts;

ProductID,ProductName,ProductCategory
209,Mouse,Accessories
206,Smartwatch,Electronics
203,Tablet,Electronics
205,Gaming Console,Electronics
201,Laptop,Electronics
202,Smartphone,Electronics
207,Monitor,Accessories
208,Keyboard,Accessories
204,Headphones,Accessories


In [0]:
INSERT INTO sales_scd.Orders (
    OrderID,
    OrderDate,
    CustomerID,
    CustomerName,
    CustomerEmail,
    ProductID,
    ProductName,
    ProductCategory,
    RegionID,
    RegionName,
    Country,
    Quantity,
    UnitPrice,
    TotalAmount
)
VALUES
(1, '2024-02-11', 101, 'Alice Johnson', 'alice@example.com', 201, 'Gaming Laptop', 'Electronics', 301, 'North America', 'USA', 2, 800, 1600),
(2, '2024-02-12', 102, 'Bob Smith', 'bob@example.com', 230, 'Airpods', 'Electronics', 302, 'Europe', 'Germany', 1, 500, 500)

num_affected_rows,num_inserted_rows
2,2


In [0]:
CREATE OR REPLACE VIEW sales_scd.view_DimProducts 
AS
SELECT DISTINCT(ProductID) AS ProductID, ProductName, ProductCategory FROM sales_scd.orders
WHERE OrderDate>'2024-02-10'

In [0]:
SELECT * FROM sales_scd.view_DimProducts;

ProductID,ProductName,ProductCategory
201,Gaming Laptop,Electronics
230,Airpods,Electronics


## MERGE - SCD TYPE 1

In [0]:
MERGE INTO sales_scd.DimProducts AS target
USING sales_scd.view_DimProducts AS source
ON target.ProductID = source.ProductID
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
select * from sales_scd.DimProducts;

ProductID,ProductName,ProductCategory
209,Mouse,Accessories
206,Smartwatch,Electronics
203,Tablet,Electronics
205,Gaming Console,Electronics
202,Smartphone,Electronics
207,Monitor,Accessories
208,Keyboard,Accessories
204,Headphones,Accessories
230,Airpods,Electronics
201,Gaming Laptop,Electronics
